In [13]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import tikzplotlib
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import os
import time
from qiskit.aqua.algorithms import NumPyEigensolver
from typing import List, NamedTuple
from qiskit.chemistry.transformations import FermionicTransformation, FermionicTransformationType, FermionicQubitMappingType
import pylab
import os
import time
from typing import List, NamedTuple

import matplotlib.pyplot as plt
import numpy as np
from IPython import display
from qiskit import QuantumCircuit
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import PauliExpectation
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.operators.converters import AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit import Parameter
from qiskit.providers.aer import QasmSimulator, AerProvider
from qiskit.providers.aer.noise import NoiseModel
from qiskit.quantum_info import Statevector
from qiskit.test.mock import *
from qiskit.visualization import plot_error_map

In [14]:
#plt.rcParams["figure.dpi"] = 75
plt.style.use("ggplot")

In [15]:
distances = np.linspace(0.05, 2.5, 40)
print(distances)

[0.05       0.11282051 0.17564103 0.23846154 0.30128205 0.36410256
 0.42692308 0.48974359 0.5525641  0.61538462 0.67820513 0.74102564
 0.80384615 0.86666667 0.92948718 0.99230769 1.05512821 1.11794872
 1.18076923 1.24358974 1.30641026 1.36923077 1.43205128 1.49487179
 1.55769231 1.62051282 1.68333333 1.74615385 1.80897436 1.87179487
 1.93461538 1.9974359  2.06025641 2.12307692 2.18589744 2.24871795
 2.31153846 2.37435897 2.43717949 2.5       ]


In [16]:
basis_sets = ['sto3g', '631g', '321g']
map_types = ['JORDAN_WIGNER', 'PARITY', 'BRAVYI_KITAEV']
threshold = 0.00000001

In [17]:
def get_qubit_hamiltonian(d, basis_set, map_type):
    """
    Obtain the qubit operators from the fermionic operators after second quantization

    Args:
        d: inter atomic distance
        basis_set: computational chemistry basis set
        map_type: encoding method

    Returns: weighted Pauli sum

    """
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(d) ,
                         unit=UnitsType.ANGSTROM,
                         charge=0,
                         spin=0,
                         basis=basis_set)
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    repulsion_energy = molecule.nuclear_repulsion_energy
    fermionic_op = FermionicOperator(h1,h2)
    qubit_op = fermionic_op.mapping(map_type=map_type, threshold=threshold)
    return qubit_op, repulsion_energy

In [10]:
qubit_ops = {}
for set in basis_sets:
    qubit_ops["qubitOp_{0}".format(set)],_ = get_qubit_hamiltonian(0.735, set,'JORDAN_WIGNER')

In [11]:
for label in qubit_ops:
    print("\n --- {} ---".format(label),qubit_ops[label])



 --- qubitOp_sto3g --- Representation: paulis, qubits: 4, size: 15

 --- qubitOp_631g --- Representation: paulis, qubits: 8, size: 185

 --- qubitOp_321g --- Representation: paulis, qubits: 8, size: 185


In [24]:
def exact_eigensolver(distance, map_type, basis_set='sto6g'):
    """
    Calculate the eigenvalue using NumPy eigensolver method

    Args:
        d: inter atomic distance
        basis_set: computational chemistry basis set
        map_type: encoding method

    Returns: exact eigenvlaue

    """
    qubit_op, repulsion = get_qubit_hamiltonian(distance,basis_set, map_type)
    result = NumPyEigensolver(qubit_op).run()
    exact_energy = np.real(result.eigenvalues) + repulsion
    return exact_energy

In [25]:
exact_energies  = []

In [ ]:
for d in distances:
    exact_energies.append(exact_eigensolver(d,'JORDAN_WIGNER'))
print("completed")

In [41]:
print(exact_energies)

[array([7.9454801]), array([2.09682804])]


In [ ]:
plt.plot(distances, exact_energies, label=r'Exact energies')
plt.xlabel('Interatomic distance')
plt.ylabel('Energy')
plt.title(r'$H_2$ Ground State Energy in different basis sets')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

#tikzplotlib.save("test.tex")